In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
import time as t
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,mean_absolute_percentage_error

In [2]:
train = pd.read_csv('adulttrain.txt')
train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
train.apply(lambda x:np.sum(x==' ?'))

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
Class                0
dtype: int64

In [4]:
mask1 = train["workclass"] != " ?"
mask2 = train["native-country"] != " ?"
mask3 = train["occupation"] != " ?"

train_v2 = train[(mask1& mask2 & mask3 )]
train_v2

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [5]:
#確認各欄位無問號資料
train_v2.apply(lambda x:np.sum(x==' ?'))

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
Class             0
dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
obj_col = ['workclass', 'education', 'marital-status', 'occupation',
           'relationship', 'race', 'sex', 'native-country', 'Class']

encoder = LabelEncoder()
data_le = pd.DataFrame(train_v2)

# 將類別欄位進行編碼
for col in train_v2[obj_col]:
    train_v2[col] = encoder.fit_transform(data_le[col])
train_v2

/var/folders/gr/xv746yg140nb4_d6mszp6jb80000gn/T/ipykernel_9670/701424537.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_v2[col] = encoder.fit_transform(data_le[col])
/var/folders/gr/xv746yg140nb4_d6mszp6jb80000gn/T/ipykernel_9670/701424537.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_v2[col] = encoder.fit_transform(data_le[col])
/var/folders/gr/xv746yg140nb4_d6mszp6jb80000gn/T/ipykernel_9670/701424537.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Class
0,39,5,77516,9,13,4,0,1,4,1,2174,0,40,38,0
1,50,4,83311,9,13,2,3,0,4,1,0,0,13,38,0
2,38,2,215646,11,9,0,5,1,4,1,0,0,40,38,0
3,53,2,234721,1,7,2,5,0,2,1,0,0,40,38,0
4,28,2,338409,9,13,2,9,5,2,0,0,0,40,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,2,257302,7,12,2,12,5,4,0,0,0,38,38,0
32557,40,2,154374,11,9,2,6,0,4,1,0,0,40,38,1
32558,58,2,151910,11,9,6,0,4,4,0,0,0,40,38,0
32559,22,2,201490,11,9,4,0,3,4,1,0,0,20,38,0


# 測試集資料前處理

In [7]:
import pandas as pd
import numpy as np
test = pd.read_csv('adulttest.txt')
test

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [8]:
test.apply(lambda x:np.sum(x==' ?'))

age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
Class               0
dtype: int64

In [9]:
mask1 = test["workclass"] != " ?"
mask2 = test["native-country"] != " ?"
mask3 = test["occupation"] != " ?"

test_v2 = test[(mask1& mask2 & mask3 )]
test_v2

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16275,33,Private,245211,Bachelors,13,Never-married,Prof-specialty,Own-child,White,Male,0,0,40,United-States,<=50K.
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K.
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K.
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K.


In [10]:
#確認各欄位無問號資料
train_v2.apply(lambda x:np.sum(x==' ?'))

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
Class             0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
obj_col = ['workclass', 'education', 'marital-status', 'occupation',
           'relationship', 'race', 'sex', 'native-country', 'Class']

encoder = LabelEncoder()
data_le = pd.DataFrame(test_v2)

# 將類別欄位進行編碼
for col in test_v2[obj_col]:
    test_v2[col] = encoder.fit_transform(data_le[col])
test_v2

/var/folders/gr/xv746yg140nb4_d6mszp6jb80000gn/T/ipykernel_9670/4278847271.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_v2[col] = encoder.fit_transform(data_le[col])
/var/folders/gr/xv746yg140nb4_d6mszp6jb80000gn/T/ipykernel_9670/4278847271.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_v2[col] = encoder.fit_transform(data_le[col])
/var/folders/gr/xv746yg140nb4_d6mszp6jb80000gn/T/ipykernel_9670/4278847271.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Class
0,25,2,226802,1,7,4,6,3,2,1,0,0,40,37,0
1,38,2,89814,11,9,2,4,0,4,1,0,0,50,37,0
2,28,1,336951,7,12,2,10,0,4,1,0,0,40,37,1
3,44,2,160323,15,10,2,6,0,2,1,7688,0,40,37,1
5,34,2,198693,0,6,4,7,1,4,1,0,0,30,37,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16275,33,2,245211,9,13,4,9,3,4,1,0,0,40,37,0
16276,39,2,215419,9,13,0,9,1,4,0,0,0,36,37,0
16278,38,2,374983,9,13,2,9,0,4,1,0,0,50,37,0
16279,44,2,83891,9,13,0,0,3,1,1,5455,0,40,37,0


In [12]:
x_train_feature = train_v2.drop(labels=['hours-per-week'],axis=1)
x_train_class = train_v2['hours-per-week'].values

In [13]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(x_train_feature)
x_scaled = scaler.transform(x_train_feature)
x_train_final=pd.DataFrame(x_scaled,columns=x_train_feature.columns)
x_train_final

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,Class
0,0.301370,0.833333,0.043338,0.600000,0.800000,0.666667,0.000000,0.2,1.0,1.0,0.021740,0.0,0.95,0.0
1,0.452055,0.666667,0.047277,0.600000,0.800000,0.333333,0.230769,0.0,1.0,1.0,0.000000,0.0,0.95,0.0
2,0.287671,0.333333,0.137244,0.733333,0.533333,0.000000,0.384615,0.2,1.0,1.0,0.000000,0.0,0.95,0.0
3,0.493151,0.333333,0.150212,0.066667,0.400000,0.333333,0.384615,0.0,0.5,1.0,0.000000,0.0,0.95,0.0
4,0.150685,0.333333,0.220703,0.600000,0.800000,0.333333,0.692308,1.0,0.5,0.0,0.000000,0.0,0.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,0.136986,0.333333,0.165563,0.466667,0.733333,0.333333,0.923077,1.0,1.0,0.0,0.000000,0.0,0.95,0.0
30158,0.315068,0.333333,0.095589,0.733333,0.533333,0.333333,0.461538,0.0,1.0,1.0,0.000000,0.0,0.95,1.0
30159,0.561644,0.333333,0.093914,0.733333,0.533333,1.000000,0.000000,0.8,1.0,0.0,0.000000,0.0,0.95,0.0
30160,0.068493,0.333333,0.127620,0.733333,0.533333,0.666667,0.000000,0.6,1.0,1.0,0.000000,0.0,0.95,0.0


In [14]:
x_test_feature = test_v2.drop(labels=['hours-per-week'],axis=1)
x_test_class = test_v2['hours-per-week'].values

In [15]:
scaler = MinMaxScaler(feature_range=(0, 1)).fit(x_test_feature)
x_scaled = scaler.transform(x_test_feature)
x_test_final=pd.DataFrame(x_scaled,columns=x_test_feature.columns)
x_test_final

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,Class
0,0.109589,0.333333,0.144430,0.066667,0.400000,0.666667,0.461538,0.6,0.50,1.0,0.000000,0.0,0.948718,0.0
1,0.287671,0.333333,0.051677,0.733333,0.533333,0.333333,0.307692,0.0,1.00,1.0,0.000000,0.0,0.948718,0.0
2,0.150685,0.166667,0.219011,0.466667,0.733333,0.333333,0.769231,0.0,1.00,1.0,0.000000,0.0,0.948718,1.0
3,0.369863,0.333333,0.099418,1.000000,0.600000,0.333333,0.461538,0.0,0.50,1.0,0.076881,0.0,0.948718,1.0
4,0.232877,0.333333,0.125398,0.000000,0.333333,0.666667,0.538462,0.2,1.00,1.0,0.000000,0.0,0.948718,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15055,0.219178,0.333333,0.156895,0.600000,0.800000,0.666667,0.692308,0.6,1.00,1.0,0.000000,0.0,0.948718,0.0
15056,0.301370,0.333333,0.136723,0.600000,0.800000,0.000000,0.692308,0.2,1.00,0.0,0.000000,0.0,0.948718,0.0
15057,0.287671,0.333333,0.244762,0.600000,0.800000,0.333333,0.692308,0.0,1.00,1.0,0.000000,0.0,0.948718,0.0
15058,0.369863,0.333333,0.047666,0.600000,0.800000,0.000000,0.000000,0.6,0.25,1.0,0.054551,0.0,0.948718,0.0


In [16]:
Start_time = t.time()

xgb = XGBRegressor(
   n_estimators=100,
   reg_lambda=1,
   gamma=0,
   max_depth=3
)
xgb.fit(x_train_final, x_train_class)
y_pred = xgb.predict(x_test_final)

End_time = t.time()

elapsed_time = End_time - Start_time

In [17]:
print('MAPE',mean_absolute_percentage_error(x_test_class,y_pred))##
print('RMSE',np.sqrt(mean_squared_error(x_test_class, y_pred)))##
print('R2',r2_score(x_test_class, y_pred))##
print(f'所需的計算時間: {elapsed_time} 秒')

MAPE 0.2605709440019268
RMSE 10.364476894500903
R2 0.2617132051927985
所需的計算時間: 0.19356012344360352 秒
